<img src="https://github.com/christopherhuntley/DATA6510/blob/master/img/Dolan.png?raw=true" width="180px" align="right">

# **DATA 6510**
# **Lesson 8: SQL Extensions and Alternatives** 
_NoSQL, Window Functions, and Collection Types_

## **Learning Objectives**
### **Theory / Be able to explain ...**

- NoSQL technologies like Key-Value Stores, Document Stores, and Graph Databases
- Recent SQL extensions geared towards analytical applications 

### **Skills / Know how to ...**
- Use window functions to work with timestamped data
- Use SQL arrays and structs to work with repeating groups and hierarchical data structures
--------

## **BIG PICTURE: Implementation Tradeoffs**
In this lesson we will explore various alternatives to the traditional **rowstore** database. Some of the alternatives will drop SQL itself, which would seem like heresy. However, there are actually some important use cases where one might not want to use a SQL-based solution. Even when SQL is the right decision there are plenty of options to choose from that fit some use cases better than others. 

In Lesson 5 we discussed four basic design priorities:
- Minimizing Storage Space
- Maximizing Calculation Speed
- Maximizing Coherency
- Minimizing Data Corruption Risk

These tradeoffs between these priorities exist regardless of the technology used. That's why they are *design* tradeoffs. However, there is more to database systems than design. Sometimes we have to broaden our scope to consider other alternatives. 

Here are four considerations that underlie any database technology decision:
- **Flexibility / Developer Experience**  
  How easy is it for developers to learn and use the technology? It may make sense to use technology that is closer to what your programmers use everyday. 
- **Scalability / Performance Speed and Cost**  
  There is a natural tradeoff between speed and cost. Technology that works at Big Data Scale makes that tradeoff explicit. How much does each GB of storage cost? How about each GB/sec of query throughput? If we are willing to wait a little longer for each query can we save some money? 
- **Consistency / Timeliness**  
  Data is worthless if it is not available when you need it. If we don't need data to be instantaneously available, how long can we wait? If we need the data right now, then how tolerant are we of anomalies and other imperfections? If we use data replication to speed up access, do we need all copies to be 100% consistent? 
- **Technical Maturity / Technical Debt**  
  Database technology is always evolving, with new solutions coming out all the time. The newest technology may score well on the above considerations but also may come with bugs and other problems that need to be worked out. Meanwhile, older technology may be rock solid but also may limit your choices going forward. There is always the risk of being stuck with obsolete technology while your competition is beating you with something newer. 

We will start with the first two considerations, developer experience and performance, which are generally used as the rationale for NoSQL technologies that don't (necessarily) adhere to the relational database model. Then we will follow up with strategies that can further improve the speed and cost performance for any technology, providing you are able to make the right consistency and maturity tradeoffs.

 

---
## **NoSQL Databases**
The term "NoSQL" was first coined in the late 1990s and gained popularity among application programmers about a decade later. It refers to databases that do not rely on the relational model. That does not mean that SQL (or some close approximation) isn't used, but rather, that NoSQL systems extend beyond the traditional relational model. 

For this reason some interpret NoSQL as "Not only SQL" rather than exclusively no use of SQL at all. In fact, each of the NoSQL technologies surveyed here *could* in fact be implemented in SQL, and we will try to use relational models to explain how each relates to SQL and how it extends beyond it. 

Why would we even need to go beyond SQL? 
- **Developer Experience (DX):** As we have discussed before, there is a natural impedance mismatch between a declarative language like SQL and a more imperative application development language like Python, Java, or JavaScript. NoSQL technologies remove much of the discomfort some programmers feel when using SQL.
- **Performance:** Selectively relaxing the rules of the relational model can sometimes bring speed and cost benefits that outweigh the integrity protections of the relational model. 

For each of the models below we will discuss how it differs from the standard relational model, potential DX or performance benefits, and the most common usage scenarios. 

### **Key-Value Stores**
Key-Value (KV) stores are most useful when a table is very sparse, like this section of the NBA PlayLog data set. 

![Sparse Table](https://github.com/christopherhuntley/DATA6510/raw/master/img/L11_Sparse_Table.png)

If most values in a table are blank, then why waste time and effort recording them in rows and columns. Instead, just store the data you actually have, tagged to suit however you may need to retrieve it. 

In relational terms, a KV store is just a single two column table   
`kv_store(`**`key`**`, value)`

where
- **`key`** is a unique index
- `value` is a datum to be stored

Usually the data type of the `value` is either encoded in the data itself (e.g., like i2'15' or s5'Steve`), encoded in the key, or assumed to be text. 

> **Heads Up:** We have already seen an example in Lesson 5. The Entity-Attribute-Value model can, with the right modeling conventions, be seen as a kind of KV store. The key would be a composite of the entity and attribute. 

You may be wondering how we can replace a two-dimensional table with rows and columns with a one dimensional KV store. We do it exactly like we would with composite indexes in the relational model, with the row and column encoded in the key. It's all in the patterns we use when constructing the keys.

For example, the following could be used to record the `assist`,`away`, etc. columns as key-value pairs:

| **Key** | **Value**|
| --- | ---|
| 2:away | Derrick Favors |
| 2:home | Marc Gasol |
| 18:opponent | Kyle Lowry |
| 20:num | 1 |
| 21:num | 2 |

KV Stores are commonly used for creating data caches for the web. Here, for example, is the data that Colab is keeping about *this page* (somewhat redacted) while I have it open in my web browser : 
![](https://github.com/christopherhuntley/DATA6510/raw/master/img/L11_Colab_Local_Storage.png)

Similar caching technology is used by web servers to minimize reads from the disk storage. If the same CSS is used on every page then why read it from disk every time? The server doesn't. Instead it serves the CSS from a highly-optimized, in-memory KV cache. 

#### **Wide-Column KV Stores**
Wide-Column data stores extend the KV model to allow data storage as multiple columns:
- The keys are just like any other KV store.
- The columns are encoded like a SQL `STRUCT` (or JSON object), with each column having a name and a value. 

The columns for each key may vary, like this:

| **Key** | **Value**|
| --- | ---|
| 2 | {away:Derrick Favors, home: Marc Gasol} |
| 18 | {opponent:Kyle Lowry} |
| 20 | {num:1} |
| 21 | {num:2} |

The effect is to condense the already very space-efficient key-value model by eliminating overlapping keys. In this case we eliminated a row of data storage by sharing the key `2` for the `away` and the `home` columns. 

#### **Summary**

- Pros
  - very fast storage and retrieval (when applicable)
  - compact storage 
  - programmer-friendly
- Cons
  - schema by convention instead of rules
  - potential for schema drift as new key types proliferate
  - no FKs or analogous way to integrate data across keys
- When to use
  - with sparse or highly volatile data where the keys need to be flexible
  - as local data stores in application development
  - when caching data to speed up application performance
- Example Products
  - [Varnish](https://varnish-cache.org/)
  - Nginx
  - Squid
  - Memcached
  - redis
  - AWS DynamoDB

### **Document Stores**
Document stores build on the KV model to construct arbitrarily complex (and data rich) databases of *semi-structured* data. Semi-structured data has a schema (so we can interpret and process it), just like any other data model, but not one we can know about in advance. Specifically, in a document store each document (roughly equivalent to a table row) can have its own schema, structuring the data however it likes. We can then only know a document schema *after* opening the document, whereas for the relational model all table schemas are known *before* doing any DML operation. 

The classic example of a document storage format is that used by Microsoft Word, which acts as a *container* for components (blocks of text, titles, images, tables, etc.). One can insert just about anything inside an MS Word file (even malware, unfortunately). MS Word will then compose and render the components as documents in real time so end users can read and edit the data contained inside. 

A more relevant example is JSON, which has become the de facto standard format for transmitting data over the web. Like with an MS Word document, a JSON object or list acts as a container, into which we can insert ... JSON objects and lists. The result is a tree structure, with objects and lists nested inside each other to some unspecified depth. 

Here, for example, is what a Colab notebook (this one, in fact) looks like when you open it in a text editor:
![Colab as JSON](https://github.com/christopherhuntley/DATA6510/raw/master/img/L11_Colab_As_JSON.png)

Yes, the `ipynb` file format is really just highly structured JSON. Here's the same JSON in a nicer, pretty printed format:
![Prettified JSON](https://github.com/christopherhuntley/DATA6510/raw/master/img/L11_Pretty_JSON.png)

A Colab Notebook is mostly a list of "cells" $-$ look for it in the screenshot $-$ where each cell has a `cell_type`, `metadata`, and `source`. The `source` is whatever we typed into the cell. 

We will come back to JSON in the **Pro Tips** section later in this lesson. 

#### **Summary**
- Pros
  - little or no impedance mismatch for programmers, especially when using JavaScript
  - very compact, especially for semi-structured data
- Cons
  - same as KV stores; complex queries are especially difficult
  - schema on read complicates app design and development; potentially buggy
- When to use
  - for local storage or web transmission of data
  - for complex hierarchically-structured data, where documents are composed of nested components
  - When storing "objects" in relational databases
- Example Products
  - CouchDB
  - mongoDB
  - AWS DynamoDB
  - Google Cloud Firestore

### **Graph Databases**
A graph database organizes data into three kinds of structures:
- A set of **nodes** that represent entities within the domain
- A set of **edges** (or arcs) that connect nodes to represent relationships
- **Properties** that represent the attributes of each entity or relationship

| ![Neo4J Screenshot](https://github.com/christopherhuntley/DATA6510/raw/master/img/L11_Neo4J_Screenshot_Annotated.png) |
|:---:|
| Original Image Source: [*Graph Databases, Linked Data, RDF, and the Semantic Web Wasteland*]( https://medium.com/@eisenzopf/graph-databases-linked-data-rdf-and-the-semantic-web-wasteland-69e9f4347a5b) |


On a superficial level, graph databases are a lot like relational databases, with nodes = entities, properties = attributes, and edges = relationships.  A graph database, however, is much more flexible in how it defines these things: 
- Since we are not using relations, there is no need to separate the entities by type. If we like, **we can treat each node as a unique type, with its own distinct properties (kept as Key-Value pairs).** 
- Whereas a relational database treats relationships as being between entity types, graph databases define them on specific node pairs. **Any pair of nodes can be connected by an edge, regardless of type.** 
- While each node has a unique identifier, there is no concept of type-based foreign keys. Instead, **each edge is treated like an entity with a unique identifier.** The edge then has two implicit properties to reference the nodes it connects. 

With all that said, a graph database *can* be mapped into a very specific relational model where:
- There are three types of entities: nodes, edges, and properties.
- Edge entities are directional, with each having a start node and an end node.
- Properties are associated with nodes or edges as needed.

So if the graph database is really just a special case of the relational model, then what's the point? For some applications a graph database can be much, much faster than a relational database. For example, consider the database behind the [Six Degrees of Kevin Bacon](https://en.wikipedia.org/wiki/Six_Degrees_of_Kevin_Bacon) game, where each working actor has a *Bacon Number*, BN, that represents the number of edges needed to connect them to Kevin Bacon. Kevin Bacon's BN is 0. Anyone who has appeared in a movie with him has a BN of 1. Anyone who has appeared with anyone whose BN=1 has a BN of 2, etc. The game is to guess an obscure actor's BN, with super bonus points for finding a movie actor *anywhere in the world* without a BN. 

Let all the movie credits in the world be stored in a single table called `credits`, with one row per actor and movie. To keep things as simple as possible we will assume that 
- Each actor and each movie has a unique identifier (`actor_id` and `movie_id`).
- We already know the `actor_id` identifiers for Kevin Bacon (`kevin_bacon_id`) and the other actor in question (`other_actor_id`).

Then to find all people with BN=1 we would join the credits table with itself one time:

```sql
-- Check for BN=1
SELECT distinct c1.actor_id 
FROM credits AS c0 
    JOIN credits AS c1 USING (movie_id)
WHERE c0.actor_id = kevin_bacon_id AND c1.actor_id = other_actor_id
```
(If we want to look for a particular actor, we can just include them as `c1.actor_id` in the `WHERE` clause. If there are no results then the actor's BN is greater than 1.) 

To find all the movies that BN=1 actors appeared in would we add in another join:
```sql
SELECT distinct c2.movie_id 
FROM credits AS c0 
    JOIN credits AS c1 ON (c0.movie_id = c1.movie_id)
    JOIN credits AS c2 ON (c1.actor_id = c2.actor_id)
WHERE c0.actor_id = kevin_bacon_id
```

We can then repeat the process with another join to get the BN=2 actors. To get the BN=3 actors we would add on two more joins to the chain. If we repeat the process out to BN=6 (the theoretical maximum for a working movie actor) then we would have 11 joins in the chain. 

```sql
SELECT distinct c11.actor_id 
FROM credits AS c0 
    JOIN credits AS c1 ON (c0.movie_id = c1.movie_id) # BN=1

    JOIN credits AS c2 ON (c1.actor_id = c2.actor_id)
    JOIN credits AS c3 ON (c2.movie_id = c3.movie_id) # BN=2

    JOIN credits AS c4 ON (c3.actor_id = c4.actor_id)
    JOIN credits AS c5 ON (c4.movie_id = c5.movie_id) # BN=3

    JOIN credits AS c6 ON (c5.actor_id = c6.actor_id)
    JOIN credits AS c7 ON (c6.movie_id = c7.movie_id) # BN=4

    JOIN credits AS c8 ON (c7.actor_id = c8.actor_id)
    JOIN credits AS c9 ON (c8.movie_id = c9.movie_id) # BN=5

    JOIN credits AS c10 ON (c9.actor_id = c10.actor_id)
    JOIN credits AS c11 ON (c10.movie_id = c11.movie_id) # BN=6
WHERE c0.actor_id = kevin_bacon_id AND c11.actor_id = other_actor_id
```

Running up to six queries like this seems ugly but still plenty doable, until you realize that the `credits` table might have 50 million rows. **Even with supercomputer hardware the query could take virtually forever.** With a graph database and a well-tuned search algorithm, however, we can find the specific sequence of edges connecting any actor to any other in less than a second. We can make it *even faster* if we assume that one of the actors is Kevin Bacon. 

The secret is in the specificity of the graph database model. We don't need to explore all possible actor-to-movie-to-actor connections. Instead we only need to follow the connections along the shortest path, which can be found fairly quickly via dynamic programming. If we want it even faster we can try A* search with an "explore big cast movies first" strategy. 

The achilles heel of the graph database model is bulk computation. If we need to do the same repetitive operation millions of times, perhaps concurrently, then a relational or perhaps a document database is likely a better fit, depending on the operation in question. Graph databases are designed for representation and searching, and they do those things very well, but not so much for statistical modeling or similar table-oriented applications. 

#### **Summary**
- Pros
  - flexible data model, with nodes, entities, and properties as first-class entity types
  - scales well for associative datasets with lots of relationships
- Cons
  - same as KV stores; complex queries are especially difficult 
  - while searchable, lacks most of the computational power of the relational model
  - treating each edges and property as a separate entity may not be space efficient
- When to use
  - whenever the data is naturally represented as nodes, edges, and properties
  - for network-focused apps like customer relationship management, road navigation, certain search engine operations, and file systems
  - for semantic maps (e.g., in text analytics) where the relationships represent interactions, forces, hierarchies, etc.
- Example Products
  - Neo4j
  - RedisGraph
  - AWS Neptune


---
## **Window Functions: How to work with longitudinal data**

Longitudinal data, where the meaning of a given fact is determined relative to the ones before it, is inherently problematic. The complexity is subtle, practically invisible to untrained eyes, but it can nonetheless wreak havoc on even the best data model. 

The relational data model is designed to capture instantaneous snapshots of whatever is being tracked, adding one row at a time. 
```sql
INSERT INTO table (columns)
  rows;
```

Similarly, `SELECT` queries return data one row at a time. That makes it nice and symmetric, with rows in and rows out. 

Now for the subtle part. If we want to include a calculated column, say the difference between two other columns in the same row, then we'd do something like this:
```sql
SELECT column_b - column_a AS b_minus_a 
FROM ... ;
```
That seems simple enough. We can also, of course, use `GROUP BY` to calculate subtotals within a given column if we like:
```sql
SELECT sum(column_a) as subtotal_a
FROM ...
GROUP BY column_b;
```
In both cases we are performing a calculation over *nonoverlapping* groups of rows and columns (i.e., either within each row or aggregated within predefined groups of rows).

However, what if we need to calculate differences between each row and *the one above it*? Then it gets more complicated, *because the groups overlap*. Not even `GROUP BY` can help us with that, unless we somehow make each row and the one above it a group, which seems highly inefficient at best. We'll instead have to dig deeper into our bag of tricks: 
- Join a table to itself using some sort of lag logic on the keys:
```
SELECT t2.column_a - t1.column_a AS diff_a_lag_1
FROM table AS t1 JOIN table AS t2 ON (t1.id = t2.id + 1)
```
- Use Python or another programming language to do the work. pandas has the `shift()` method just for such a purpose. You merge one table with a shifted version of itself. If you go this route, of course, you will need SQL to read the input data and then write the calculations back to the database afterwards. That's a nontrivial solution to say the least.  
- Use a SQL **window function**, which operates vertically within a column instead of laterally within a row. 

For most use cases, window functions will be your best choice. 

Window functions are used in the `SELECT` and `ORDER BY` clauses to aggregate longitudinal data *without a `GROUP BY` clause*. Instead of the traditional grouping operation, the expression being calculated specifies a *sliding window* of rows, anchored by the current row. The syntax is 
```sql
expression OVER ( window_selector )
```
The `window_selector` specifies what rows to include and can use pretty sophisticated selection criteria like any of the following:
- X rows before and Y rows after the current row
- All rows before the current row
- All rows where column X is within some range of Z in the current row
- X rows before and Y rows after, sorted by the value of Z

The window function partitions the rows (much like `GROUP BY`) based on the given criteria and then calculates the expression based on the current row. The expression is almost always an aggregation function like `SUM()` or `AVG()`, of which there are too many to list here. There are even special ones like [`STRING_AGG()`](https://cloud.google.com/bigquery/docs/reference/standard-sql/functions-and-operators#string_agg) that work with non-numeric data types. 

Here, for example, we are calculating the cumulative minutes for each NBA player lineup throughout a season of play.  





In [ ]:
# load the bigquery magics extension
%load_ext google.cloud.bigquery
%load_ext google.colab.data_table

# authenticate for GCP access
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery
Authenticated


In [ ]:
%%bigquery --project nba-lineup-facts
SELECT 
    game_id, 
    date,
    lineup, 
    sum(play_length_mins) OVER 
      (PARTITION BY lineup ORDER BY game_id ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW ) 
      AS cum_mins
FROM `nba-lineup-facts.lineup_facts.game_facts_all` 
WHERE year = 2020
ORDER BY lineup, date
LIMIT 100

,game_id,date,lineup,cum_mins
0,21900415,2019-12-18,"[""D'Angelo Russell"", 'Damion Lee', 'Draymond G...",0.083333
1,21900665,2020-01-22,"[""D'Angelo Russell"", 'Damion Lee', 'Draymond G...",0.700000
2,21900679,2020-01-24,"[""D'Angelo Russell"", 'Damion Lee', 'Draymond G...",4.233333
3,21900757,2020-02-05,"[""D'Angelo Russell"", 'Damion Lee', 'Draymond G...",13.450000
4,21900757,2020-02-05,"[""D'Angelo Russell"", 'Damion Lee', 'Draymond G...",1.733333
...,...,...,...,...
95,21900602,2020-01-14,"[""D'Angelo Russell"", 'Draymond Green', 'Glenn ...",2.666667
96,21900588,2020-01-12,"[""D'Angelo Russell"", 'Draymond Green', 'Glenn ...",1.500000
97,21900602,2020-01-14,"[""D'Angelo Russell"", 'Draymond Green', 'Glenn ...",2.183333
98,21900076,2019-11-01,"[""D'Angelo Russell"", 'Draymond Green', 'Glenn ...",3.216667


The window selector has three parts:
```sql 
PARTITION BY lineup 
ORDER BY game_id 
ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW 
```
- `PARTITION BY` works like a `GROUP BY` to split the rows into groups/partitions.
- `ORDER BY` does as expected, but always with respect to the partition.
- `ROWS` specifies the width of the window; in this case that means everything up to the current row in the current partition (lineup).

There are lots of variations to play with. For details read the docs for your database vendor. The SQL standard defines the basics like `PARTITION BY`, `ORDER BY`, and `ROWS` but there are some differences for more advanced use cases.  

> **Heads Up**: Although window functions were included in SQL:2003, most of the vendors only recently implemented the standard.  Check to see if your DBMS supports them before trying to use them.  

---
## **Collection Types**

In lesson 5 we learned that entities should be atomic, indivisible through normalization. Among other things this means that attributes should be singular, represeting just one fact about a given entity. Plural attributes, meanwhile, should be normalized out into one-to-many relationships to new entity types. 

But what if we just didn't bother to do that? 

The `ARRAY` and `STRUCT` collection types (details below) were added to the SQL standards in a wave of enthusiasm for object-relational features in SQL:1999 but then (like window functions) largely ignored by most database vendors until 2018. Why ignore the standard? Because collection types permit us to skirt around a fundamental law of normalization: thou shalt not use repeated fields. 

In fact, it is not a good idea to use these sorts of collection types in a transactional database, where vigilance is needed to keep the data consistent and anomaly free. However, for dimensional data warehouses, where the **dimension tables** are often denormalized anyway, they make **perfect sense**: no extra joins, no redundant columns, etc., just the relevant context in a concise format.  

> **Heads Up**: Collection types are generally **not a good fit for fact tables** because they complicate the aggregation logic. What does a **measure** (i.e., fact) mean when it can be multi-valued? How do you sum or average a list of lists? 

***Technically*, collection types do not actually violate normalization rules.** A collection is a **container** that can hold structured data. The container itself is singular, though the contents might be plural. It's like how you might keep a plastic *container* of cut celery sticks instead of storing them in a pile on the shelf. The container is *one* thing (with the sticks nested inside), but you have to open the container (unpack it) to get your snack. That's exactly how SQL `ARRAY` and `STRUCT` work. We have to use functions and special syntax to pack and unpack data. Otherwise, SQL just treats it like an image or other binary data. Of course, it *is* still a problem that we don't know what's inside the container before we unpack it but we can still say the table itself is 1NF. 

Now let's go back to the NBA data to see how it works. To keep things simple we will just focus on the lineup dimension, which in our play log data is kept in columns `a1`-`a5` and `h1`-`h5`.

   


### **The `ARRAY` Data Type**

An `ARRAY` is a list of items where:
- The items are ordered, one after another, like the rows of a table.  
- All items have the same data type. If the first item is an integer, then so is every other item. 
- There can be any number of items. 

Note that these same rules also apply to table columns, where each column has a data type, is ordered by row, and there is no limit on how many items (rows) there are. This is by design, so that the same logic can apply.  

The query below returns two rows of play log data, with each lineup as an **`ARRAY`** (represented by the square brackets `[ ]`)  

In [ ]:
%%bigquery --project nba-lineup-facts
SELECT
  [a1,a2,a3,a4,a5] as away_lineup, 
  [h1,h2,h3,h4,h5] as home_lineup
FROM `nba-lineup-facts.lineup_facts.play_log_sample`
WHERE play_id IN (86,88);

,away_lineup,home_lineup
0,"[OG Anunoby, Marc Gasol, Kyle Lowry, Fred VanV...","[Gordon Hayward, Jaylen Brown, Kemba Walker, R..."
1,"[OG Anunoby, Marc Gasol, Kyle Lowry, Fred VanV...","[Gordon Hayward, Kemba Walker, Robert Williams..."


At first glance, this looks a lot like the text strings we have been using to represent NBA lineups. To see the difference, consider these two queries.


In [ ]:
%%bigquery --project nba-lineup-facts

-- Lineup string: "['OG Anunoby','Pascal Siakam','Marc Gasol','Kyle Lowry','Fred VanVleet']"
-- Does the lineup string include the text 'Marc Gasol'?
SELECT "['OG Anunoby','Pascal Siakam','Marc Gasol','Kyle Lowry','Fred VanVleet']" LIKE '%Marc Gasol%'

,f0_
0,True


In [ ]:
%%bigquery --project nba-lineup-facts

-- Lineup ARRAY: ['OG Anunoby','Pascal Siakam','Marc Gasol','Kyle Lowry','Fred VanVleet']
-- Does the player 'Marc Gasol' appear in the lineup array? 
SELECT 'Marc Gasol' IN UNNEST(['OG Anunoby','Pascal Siakam','Marc Gasol','Kyle Lowry','Fred VanVleet'])

,f0_
0,True


Both return the same result but work very differently. The first query looks for the text `'Marc Gasol'` within a lineup string, while the second query looks for *Marc Gasol's name* in the lineup array. That's much more natural, isn't it? Further, the array version is much more flexible. We could, for example, replace every player's name with a `player_id` without generating and searching through lineup strings. 

There is another difference as well. When retrieving an `ARRAY` as a list we use the `UNNEST` function, which gets around the relational model's rule that each column contains scalar (singular) values. The stored array is a *container* for list items that needs to be *unpacked* before we can use it. The container is a singular thing that happens to hold more things.

> **Heads Up**: We can also [create arrays from subqueries using the `ARRAY()` function](https://cloud.google.com/bigquery/docs/reference/standard-sql/array_functions), which opens up lots of possibilities. Basically, any column of data that we can select or construct can be inserted into an array.  
 

### **The `STRUCT` Data Type**

A `STRUCT` (short for Structured Array) is like a one-row mini-table, with named *fields* that can mix data of various types. Some call it the swiss army knife of data types. Anything we can store in a table row (including `ARRAY` data) can also be stored in a structured array.

In [ ]:
%%bigquery --project nba-lineup-facts
SELECT
  STRUCT(a1 AS p1, a2 AS p2, a3 AS  p3,a4 AS p4,a5 AS p5) AS away_lineup,
  STRUCT(h1 AS p1, h2 AS p2, h3 AS  p3,h4 AS p4,h5 AS p5) AS home_lineup
FROM `nba-lineup-facts.lineup_facts.play_log_sample`
WHERE play_id IN (86,88);

,away_lineup,home_lineup
0,"{'p1': 'OG Anunoby', 'p2': 'Marc Gasol', 'p3':...","{'p1': 'Gordon Hayward', 'h2': 'Jaylen Brown',..."
1,"{'p1': 'OG Anunoby', 'p2': 'Marc Gasol', 'p3':...","{'p1': 'Gordon Hayward', 'h2': 'Kemba Walker',..."


Each `STRUCT` works more or less like a `SELECT` clause:
- The fields are defined like column specs, with aliases used to name the fields.
- `STRUCT` collections are created by calling a function, `STRUCT()`, that assembles the data into the structured array.
- Each field can store the result of any SQL expression. The expression is only evaluated once, just before the data is written to the field.

Again, a structured array is like a one-row table, with all the sophistication that implies. All it is missing is primary and foreign keys.

> **Heads Up**: The notation used by SQL to represent a structured array is basically identical to the Python `dict` and the JavaScript `object`. We will explore the relationship to JavaScript more in depth in Lesson 11. 

To unpack a structured array we use `column.field` dot notation. The example below packs players into home and away lineups (via a CTE `WITH` clause) and then unpacks them to recreate the `h1`-`h5` and `a1`-`a5` columns in the original play log dataset. 


In [ ]:
%%bigquery --project nba-lineup-facts
-- A named subquery with two STRUCT columns
WITH play_lineups AS  
  ( SELECT
      play_id,
      STRUCT(a1 AS p1, a2 AS p2, a3 AS  p3,a4 AS p4,a5 AS p5) AS away_lineup,
      STRUCT(h1 AS p1, h2 AS p2, h3 AS  p3,h4 AS p4,h5 AS p5) AS home_lineup
    FROM `nba-lineup-facts.lineup_facts.play_log_sample`
    WHERE play_id IN (86,88)
  )
-- Use the subquery like a table
SELECT home_lineup.p1 as h1, home_lineup.p2 as h2, home_lineup.p3 as h3, home_lineup.p4 as h4, home_lineup.p5 as h5,
       away_lineup.p1 as a1, away_lineup.p2 as a2, away_lineup.p3 as a3, away_lineup.p4 as a4, away_lineup.p5 as a5
FROM play_lineups


,h1,h2,h3,h4,h5,a1,a2,a3,a4,a5
0,Gordon Hayward,Jaylen Brown,Kemba Walker,Robert Williams III,Marcus Smart,OG Anunoby,Marc Gasol,Kyle Lowry,Fred VanVleet,Norman Powell
1,Gordon Hayward,Kemba Walker,Robert Williams III,Marcus Smart,Grant Williams,OG Anunoby,Marc Gasol,Kyle Lowry,Fred VanVleet,Norman Powell


### **Combining `ARRAY` and `STRUCT` to make complex data types**

We can combine `ARRAY` and `STRUCT` data to create arbitrarily complex data structures. The result is **trees** of data, with collections nested inside collections inside of collections ...

We saw an example in Lesson 5 when we explored JSON formatted data. The JSON fields shown below with arrowheads are collections, which have even more collections nested inside:
![](https://github.com/christopherhuntley/DATA6510/raw/master/img/L6_Space_Jam_JSON.png)

The actual JSON data, with nested arrays and structs can be found [here](https://github.com/christopherhuntley/DATA6510/blob/master/data/JSON/spacejam.json).

To show how this is done in SQL, we will explore two techniques for creating *pseudo*-tables using nested collections:
- Create each column as an `ARRAY` and then nesting them inside a `STRUCT`
- Create each row as a `STRUCT` and then nesting them inside an `ARRAY`

In both cases the result is a single value that is functionally equivalent to an entire table. 

> **Heads Up**: The examples below are not intended to be of much practical use. There are likely better ways to store tabular data, like perhaps in a table. They do, however, show the flexibility of the collection data types. Any data we can store in a table we can also store in a collection. 









#### **Nesting `ARRAY` columns inside a `STRUCT` pseudo-table**

In [ ]:
%%bigquery --project nba-lineup-facts
SELECT STRUCT(
  ARRAY(SELECT play_id FROM `nba-lineup-facts.lineup_facts.play_log_sample` WHERE play_id in (86,88)) as play_id,
  ARRAY(SELECT a1 FROM `nba-lineup-facts.lineup_facts.play_log_sample` WHERE play_id in (86,88)) as a1,
  ARRAY(SELECT a2 FROM `nba-lineup-facts.lineup_facts.play_log_sample` WHERE play_id in (86,88)) as a2,
  ARRAY(SELECT a3 FROM `nba-lineup-facts.lineup_facts.play_log_sample` WHERE play_id in (86,88)) as a3,
  ARRAY(SELECT a4 FROM `nba-lineup-facts.lineup_facts.play_log_sample` WHERE play_id in (86,88)) as a4,
  ARRAY(SELECT a5 FROM `nba-lineup-facts.lineup_facts.play_log_sample` WHERE play_id in (86,88)) as a5,
  ARRAY(SELECT a1 FROM `nba-lineup-facts.lineup_facts.play_log_sample` WHERE play_id in (86,88)) as h1,
  ARRAY(SELECT a2 FROM `nba-lineup-facts.lineup_facts.play_log_sample` WHERE play_id in (86,88)) as h2,
  ARRAY(SELECT a3 FROM `nba-lineup-facts.lineup_facts.play_log_sample` WHERE play_id in (86,88)) as h3,
  ARRAY(SELECT a4 FROM `nba-lineup-facts.lineup_facts.play_log_sample` WHERE play_id in (86,88)) as h4,
  ARRAY(SELECT a5 FROM `nba-lineup-facts.lineup_facts.play_log_sample` WHERE play_id in (86,88)) as h5 
) AS pseudo_table_by_cols

,pseudo_table_by_cols
0,"{'play_id': [86, 88], 'a1': ['OG Anunoby', 'OG..."


#### **Nesting `STRUCT` rows inside an `ARRAY` pseudo-table**

In [ ]:
%%bigquery --project nba-lineup-facts
SELECT ARRAY(
  SELECT STRUCT(play_id, a1,a2,a3,a4,a5, h1,h2,h3,h4,h5)
  FROM `nba-lineup-facts.lineup_facts.play_log_sample`
  WHERE play_id in (86,88) 
) AS pseudo_table_by_rows

,pseudo_table_by_rows
0,"[{'play_id': 86, 'a1': 'OG Anunoby', 'a2': 'Ma..."


---
## **Congratulations! You've made it to the end of Lesson 8.**

Next week we will consider alternative data models that can improve flexibility and performance. 



## **On your way out ... Be sure to save your work**.
In Google Drive, drag this notebook file into your `DATA6510` folder so you can find it next time.